# Fine Tune reward model from scratch

# TODOs:

#TODO: double-check that labels are not somehow misaligned...

#TODO set up cuda usage when running training

#TODO: find out good lora config based on paper/ other source

#TODO: check if you need to plot 

#TODO:  Edit data given by Roos such that it gives goed/fout for position...

1. Using Legal BERT with a regression head to be able to generate continuous rewards --> ask Roos from which paper that was taken

2. LoRA learns the position of the low rank adaptation matrix that is needed to finetune a model of a much higher rank

## 1. Imports, setup, and global variables

In [1]:
import torch
import pandas as pd

from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from torch.nn import functional as F
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error, mean_absolute_error

from datasets import Dataset

from peft import LoraConfig, get_peft_model, PeftModel

# import matplotlib.pyplot as plt
# import wandb



# load thing
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU.


In [2]:
FEEDBACK_TO_TRAIN_ON = 'feedback_extraction' 
FEEDBACK_TO_REMOVE = 'feedback_detection'
MODEL = "nlpaueb/legal-bert-base-uncased" # DTAI-KULeuven/robbert-2023-dutch-large --> use this for Dutch text

## 2. Dataset loading and preprocessing

In [4]:
df = pd.read_csv('C:\\Users\\jacqu\\development\\master_thesis\\RL\\roos_data.csv', sep=';') #TODO: change to proper data name once available

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 141



,file,frame_ID,frame_type,frame_text,precondition_id,precondition_text,precondition_position,response_text,prompt_config_examples,prompt_config_chain_of_thought,feedback_extraction,feedback_detection,additional_feedback
9,Interpretatie_Vw_over_besluiten_op_aanvragen_v...,87433709-4703-47ec-a236-3e064973e4c3,fact,vreemdeling,87433709-4703-47ec-a236-3e064973e4c3,vreemdeling,Artikel 1 IN Vreemdelingenwet 2024,1. Vreemdeling: Vreemdeling\r\n\r\n ...,False,True,Volledig correct,Zeer duidelijk,NaN
62,Interpretatie_Vw_over_besluiten_op_aanvragen_v...,2168d545-ef9d-4d7f-b21f-ee3fa1b0d8df,act,afwijzen aanvraag van vreemdeling tot het verl...,43fe2671-232c-4cc1-aac2-dc2855fdb796,NOT de vreemdeling dan wel de persoon bij wie ...,"Artikel 16, sectie 1 c IN Vreemdelingenwet 2024",1. De vreemdeling beschikt niet over een geldi...,False,True,Volledig correct,Zeer duidelijk,NaN
116,Participatiewet_most_recent_public.json,4d3d4ef8-e493-4596-adde-183285f22e04,act,weigeren recht op algemene bijstand,d097e347-671e-42e8-8a2c-6e7ce107bcfb,college stelt het recht op bijstand vast op sc...,"Artikel 43, sectie 1 IN Participatiewet",1. Geen recht op algemene bijstand heeft degen...,False,True,Volledig fout,Helemaal niet duidelijk,NaN
113,Participatiewet_most_recent_public.json,4d3d4ef8-e493-4596-adde-183285f22e04,act,weigeren recht op algemene bijstand,e627f4e5-5f4a-4dce-9c18-fe9775e9bc6b,bijstand wordt toegekend voor de dag waarop de...,"Artikel 44, sectie 1 IN Participatiewet",Inhoud: <tekstfragment waarin de preconditi...,False,True,Volledig fout,Helemaal niet duidelijk,NaN
66,Interpretatie_Vw_over_besluiten_op_aanvragen_v...,2168d545-ef9d-4d7f-b21f-ee3fa1b0d8df,act,afwijzen aanvraag van vreemdeling tot het verl...,e116e23f-16a6-46a5-873e-2ef818170e7b,"de vreemdeling voldoet aan de beperking, verba...","Artikel 16, sectie 1 g IN Vreemdelingenwet 2024",1. De vreemdeling beschikt niet over een geldi...,False,True,Volledig correct,Zeer duidelijk,NaN
0,Interpretatie_Vw_over_besluiten_op_aanvragen_v...,7e8d151e-a6ad-4877-85db-5eda2990ac67,fact,Onze Minister,341a8e52-ca51-42ec-b84e-f606887369e7,Onze Minister in de Vreemdelingenwet en de daa...,Artikel 1 IN Vreemdelingenwet 2024,1. Subfact: Onze Minister\r\n 2...,False,True,Grotendeels correct,Duidelijk,NaN
110,Participatiewet_most_recent_public.json,4d3d4ef8-e493-4596-adde-183285f22e04,act,weigeren recht op algemene bijstand,7e931c05-d91e-4288-8936-24fe02301a7f,belanghebbende heeft in verband met volgen ond...,"Artikel 13, sectie 2 c 1 IN Participatiewet",Inhoud: <tekstfragment waarin de preconditi...,False,True,Grotendeels correct,Helemaal niet duidelijk,NaN
133,Participatiewet_most_recent_public.json,4d3d4ef8-e493-4596-adde-183285f22e04,act,weigeren recht op algemene bijstand,e627f4e5-5f4a-4dce-9c18-fe9775e9bc6b,bijstand wordt toegekend voor de dag waarop de...,"Artikel 44, sectie 1 IN Participatiewet",1. Geen recht op algemene bijstand heeft degen...,False,True,Volledig fout,Helemaal niet duidelijk,NaN
69,Interpretatie_Vw_over_besluiten_op_aanvragen_v...,2168d545-ef9d-4d7f-b21f-ee3fa1b0d8df,act,afwijzen aanvraag van vreemdeling tot het verl...,5cb97c0b-d778-4923-be16-128176485abd,de vreemdeling in Nederland verblijf heeft geh...,"Artikel 16, sectie 1 j IN Vreemdelingenwet 2024",1. De vreemdeling beschikt niet over een geldi...,False,True,Volledig correct,Zeer duidelijk,NaN
39,Interpretatie_Vw_over_besluiten_op_aanvragen_v...,4cf6080d-b56f-4f5d-8386-3be18add6b63,fact,de reguliere verblijfsvergunning voor bepaalde...,82bf3f57-919a-4f58-b57f-a495d9b60a7a,"de nadelige gevolgen van een besluit zijn, voo...","Artikel 3:4, sectie 2 IN Algemene wet bestuurs...",1. Subfact: De aanvraag is ontvangen\r\n ...,False,True,Volledig fout,Helemaal niet duidelijk,NaN


### 2. a) Parse ratings to numeric values for MSE Loss

In [5]:
# TODO: how to parse the ratings properly here, discuss with Roos

def parse_ratings(rating):
    rating_dict = {
        "Volledig fout": 0,
        "Grotendeels fout": 1,
        "Grotendeels correct": 2,
        "Volledig correct": 3,
        "Bestaat niet in ground truth": 0, #TODO: change
        "Fout": 1,
        "Goed": 2,
    }
    return rating_dict.get(rating, None)

In [5]:
# queries = df['query'].tolist()
# answers = df['answer'].tolist()
# feedback_1 = df['feedback_extraction'].tolist()
# feedback_2 = df['feedback_detection'].tolist()

In [6]:
df[FEEDBACK_TO_TRAIN_ON] = [parse_ratings(feedback) for feedback in df[FEEDBACK_TO_TRAIN_ON]]
print("Parsed feedback for extraction:", df[FEEDBACK_TO_TRAIN_ON][:5])

Parsed feedback for extraction: 0    2.0
1    2.0
2    2.0
3    2.0
4    3.0
Name: feedback_extraction, dtype: float64


### 2. b) keep only relevant feedback column

In [7]:
dataset = Dataset.from_pandas(df)

print(dataset)

Dataset({
    features: ['file', 'frame_ID', 'frame_type', 'frame_text', 'precondition_id', 'precondition_text', 'precondition_position', 'response_text', 'prompt_config_examples', 'prompt_config_chain_of_thought', 'feedback_extraction', 'feedback_detection', 'additional_feedback'],
    num_rows: 141
})


In [8]:
dataset = dataset.remove_columns([FEEDBACK_TO_REMOVE])
dataset = dataset.rename_column(FEEDBACK_TO_TRAIN_ON, "label")

## 3. Load model with LoRA layer

In [9]:
# Load the model and the tokenizer
model_id = MODEL 
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=1) # num_labels = 1 since we want to prodict a single scalar (the rating)

# Comment: Automodel for sequence classification with num_labels=1 already has a regression head
print(model)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\jacqu\Anaconda3\envs\RL_Master_thesis\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jacqu\.cache\huggingface\hub\models--nlpaueb--legal-bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
# Define LoRA config
#TODO: find out which lora config makes sense for me
lora_config = LoraConfig(
    r=8,           # Rank of the LoRA matrices (smaller = less memory)
    lora_alpha=16, # Scaling factor (higher = stronger adaptation)
    target_modules=["query", "key", "value"], # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"  # Adjust for your task
)

# Convert the model to a PEFT (LoRA) model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Check trainable params (~0.1% of full model)

trainable params: 443,137 || all params: 109,926,146 || trainable%: 0.4031


In [11]:
sample_data = ["What is the capital of France?", "What is the largest capital in the world?"]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1067, 223, 207, 580, 210, 1335, 124, 102, 0, 0, 0], [101, 1067, 223, 207, 5601, 190, 580, 213, 207, 1727, 124, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## 4. Define Custom Trainer to be used for the task

In [ ]:
class CustomRewardTrainer(Trainer):
    def __init__(self, *args, loss_type="mse", weight_strategy=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_type = loss_type  # "mse", "huber", or custom
        self.weight_strategy = weight_strategy  # "linear", "inverse", or None

    def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
        # Extract labels (ratings) and optional sample weights
        labels = inputs.pop("labels").float()  # Shape: (batch_size)
        
        # Optional: Compute sample weights dynamically
        weights = self._get_sample_weights(labels) if self.weight_strategy else None
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits.squeeze()  # Shape: (batch_size) --> logits are the predicted rewards in this case
        
        # Custom loss calculation
        loss = self._compute_custom_loss(logits, labels, weights)
        
        return (loss, outputs) if return_outputs else loss

    def _compute_custom_loss(self, logits, labels, weights=None):
        if self.loss_type == "mse":
            loss = F.mse_loss(logits, labels, reduction="none") # --> MSE provides precise regression BUT sensitive to outliers
        elif self.loss_type == "huber":
            loss = F.huber_loss(logits, labels, reduction="none", delta=1.0) #--> balances between MSE and MAE for data that has outliers/ noise
        else:
            raise ValueError(f"Unsupported loss type: {self.loss_type}")

        # Apply sample weights if provided
        if weights is not None:
            loss = loss * weights
            loss = loss.mean()  # Normalize by mean if weights are unnormalized
        else:
            loss = loss.mean()
        
        return loss

    def _get_sample_weights(self, labels):
        """
        Generate sample weights based on rating values.
        
        
        """
        if self.weight_strategy == "linear":
            # Linear weighting (e.g., emphasize extremes)
            weights = torch.abs(labels - labels.mean()) + 1.0
        elif self.weight_strategy == "inverse":
            # Inverse frequency weighting (if ratings are skewed)
            unique, counts = torch.unique(labels, return_counts=True)
            freq = counts.float() / len(labels)
            weight_map = 1.0 / (freq + 1e-6)  # Avoid division by zero
            weights = torch.tensor([weight_map[(unique == lbl).nonzero().item()] for lbl in labels])
        else:
            weights = None
        
        return weights.to(labels.device) if weights is not None else None



    def compute_metrics(self, eval_preds):
        predictions, labels = eval_preds
        predictions = predictions.squeeze()
        
        # Regression metrics
        mse = mean_squared_error(labels, predictions)
        pearson = pearsonr(labels, predictions)[0] # Pearson correlation coefficient
        
        # Threshold accuracy --> 
        tolerance_acc = (np.abs(predictions - labels) <= 0.5).mean()
        
        return {"mse": mse, "pearson": pearson, "tolerance_acc": tolerance_acc}
    


    
    #TODO: evaluate whether the plotting should be done or whether it is redundant to add them

    # def evaluation_loop(self, *args, **kwargs):
    #     output = super().evaluation_loop(*args, **kwargs)
    #     predictions = output.predictions.squeeze()
    #     labels = output.label_ids
        
    #     # Generate plots (saved to disk or logged to W&B)
    #     plot_distributions(predictions, labels, self.state.epoch)
    #     plot_calibration(predictions, labels)
        
    #     return output

In [13]:
#TODO: debug if this is truly needed...

# add distributioncallback to trianing to evaluate 

# class DistributionCallback(TrainerCallback):
#     def on_evaluate(self, args, state, control, metrics, **kwargs):
#         # Get predictions and labels from the trainer's eval loop
#         eval_results = trainer.evaluate()
#         predictions = eval_results["eval_predictions"]
#         labels = eval_results["eval_labels"]
        
#         # Log histogram to W&B
#         wandb.log({
#             "reward_histogram": wandb.Histogram(predictions),
#             "true_ratings_histogram": wandb.Histogram(labels),
#         })


# def plot_distributions(predictions, labels, epoch):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.hist(predictions, bins=20, alpha=0.7, label="Predicted")
#     plt.title("Predicted Rewards")
    
#     plt.subplot(1, 2, 2)
#     plt.hist(labels, bins=20, alpha=0.7, label="True Ratings", color="orange")
#     plt.title("True Ratings")
    
#     plt.savefig(f"distributions_epoch_{epoch}.png")
#     plt.close()

# class PlotCallback(TrainerCallback):
#     def on_evaluate(self, args, state, control, **kwargs):
#         predictions = trainer.predict(test_dataset).predictions.squeeze()
#         labels = test_dataset["ratings"]
#         plot_distributions(predictions, labels, state.epoch)




# def plot_calibration(predictions, labels):
#     """
#     Function to check if the sdfs

    

#     """
#     bin_means = np.linspace(1, 5, num=5)  # For 1-5 ratings
#     bin_centers = []
#     empirical_means = []
    
#     for i in range(len(bin_means) - 1):
#         mask = (labels >= bin_means[i]) & (labels < bin_means[i+1])
#         if mask.sum() > 0:
#             bin_centers.append((bin_means[i] + bin_means[i+1]) / 2)
#             empirical_means.append(predictions[mask].mean())
    
#     plt.plot(bin_centers, empirical_means, marker="o")
#     plt.plot([1, 5], [1, 5], linestyle="--", color="gray")  # Ideal line
#     plt.xlabel("True Rating")
#     plt.ylabel("Predicted Reward")
#     plt.savefig("calibration_plot.png")



### 4. a) Define a custom data collator

In [14]:
# from transformers import DefaultDataCollator

# #TODO: only do this if the labels fix does not work for some reason

# class RewardDataCollator(DefaultDataCollator):
#     def __call__(self, features):

#         ratings = [f.pop("rating") for f in features]  # Removes rating from features temporarily
#         batch = super().__call__(features)
#         # Explicitly ensure rating is included
#         print(features)
#         # Re-inject ratings into the batch
#         batch["rating"] = torch.tensor(ratings, dtype=torch.float32)
#         return batch

## 5. Encode dataset

In [18]:
# TODO: combine all the relevant points from the dataframe

dataset.column_names

['file',
 'frame_ID',
 'frame_type',
 'frame_text',
 'precondition_id',
 'precondition_text',
 'precondition_position',
 'response_text',
 'prompt_config_examples',
 'prompt_config_chain_of_thought',
 'label',
 'additional_feedback']

## Comment

1. Needed for feedback extraction: precondition_text, response_text, label(rating feedback extraction)
2. Needed for feedback detection: precondition_text, precondition_position, response_text, label (rating feedback detection)
3. For the precondition position to be found well, it is a crucial for the model to find the precondition text (at least to a recognizable degree) as well, otherwise the precondition is not found at all...

In [20]:
# tokenize queries and answers together to provide proper context to reward model

#TODO: get proper columns from the 
def tokenize_fn(examples):
    combined_texts = [f"{p} {a} {r}" for p, a, r in zip(examples["precondition_text"], examples["response_text"], examples["label"])]
    return tokenizer(combined_texts, truncation=True, padding="max_length")

dataset = dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

In [21]:
print(dataset)

Dataset({
    features: ['file', 'frame_ID', 'frame_type', 'frame_text', 'precondition_id', 'precondition_text', 'precondition_position', 'response_text', 'prompt_config_examples', 'prompt_config_chain_of_thought', 'label', 'additional_feedback', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 141
})


## 6. Train reward model

In [22]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_checkpoints",
    save_strategy='steps',
    save_steps=100,
    save_total_limit=3,
    per_device_train_batch_size=8,
    learning_rate=3e-4,
    num_train_epochs=3,
    logging_steps=10,
    label_names=["labels"],
    report_to="none",
    # report_to='wandb',
)

# Initialize custom trainer
trainer = CustomRewardTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    loss_type="huber",  # Try "mse" or "huber"
    weight_strategy="linear",  # Try "linear", "inverse", or None
    # data_collator=RewardDataCollator()
)

# # add distributioncallback to trainer TODO: only integrate if relevant
# trainer.add_callback(DistributionCallback())

#train model
trainer.train()

KeyboardInterrupt: 

In [ ]:
# store final model parameters
model.save_pretrained("final_lora_adapters")

# Reload saved LoRA adapter for inference 

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model = PeftModel.from_pretrained(base_model, "final_lora_adapters")
model = model.merge_and_unload()